In [11]:
import csv
import numpy as np
from datetime import datetime
import psycopg2
import argparse

def run_queries(date, target, sender):    
    conn = psycopg2.connect(
        host="10.7.8.59",
        database="fixtransactions",
        user="scalp",
        password="QAtr@de442",
        port="5433"
    )

    query1 = f"""
    CREATE TEMP TABLE temp_table1 AS
    SELECT tag11, tag100, tag41, tag49 
    FROM fixmsg 
    WHERE tag52 >= TIMESTAMP '{date}'
    AND tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
    AND tag56 = '{target}'
    and tag49 = '{sender}'
    AND (tag35 = 'AB' or tag35 = 'D');
    """

    query2 = f"""
    CREATE TEMP TABLE temp_table2 AS
    SELECT a.tag11, a.tag52 AS cancel_52, b.tag52 AS cancelack_52, b.tag41, a.tag100
    FROM fixmsg a
    JOIN fixmsg b ON a.tag11 = b.tag41
    WHERE a.tag52 >= TIMESTAMP '{date}'
    AND a.tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
    AND b.tag52 >= TIMESTAMP '{date}'
    AND b.tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
    and a.tag49 = '{sender}'
    and b.tag56 = '{sender}'
    and b.tag49 = '{target}'
    AND b.tag35 = '8'
    AND b.tag39 = '4'
    AND (a.tag35 = 'AB' or a.tag35 = 'D')
    AND a.tag56 = '{target}';
    """

    with conn.cursor() as cursor:
        cursor.execute(query1)
        cursor.execute(query2)

        cursor.execute("SELECT * FROM temp_table2")
        query_result = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]

    conn.close()

    hash_table = {}
    for row in query_result:
        row_dict = {column_names[i]: row[i] for i in range(len(column_names))}
        hash_table[row_dict["tag11"]] = row_dict

    return hash_table

def run_analysis(date, target, hash_table):
    table2_data = list(hash_table.values())

    counts = {}
    for row in table2_data:
        tag100 = row['tag100']
        if tag100 in counts:
            counts[tag100] += 1
        else:
            counts[tag100] = 1

    result_data = []
    for row in table2_data:
        tag100 = row['tag100']
        count = counts.get(tag100, 0)
        if count > 0:
            result_data.append({
                'tag11': row['tag11'],
                'cancel_52': row['cancel_52'],
                'cancelack_52': row['cancelack_52'],
                'tag41': row['tag41'],
                'tag100': tag100,
                'counts': count
            })

    result_data = sorted(result_data, key=lambda x: (datetime.strptime(x['cancelack_52'], '%Y-%m-%d %H:%M:%S.%f') - datetime.strptime(x['cancel_52'], '%Y-%m-%d %H:%M:%S.%f')).total_seconds(), reverse=True)

    time_diff_values = [(datetime.strptime(row['cancelack_52'], '%Y-%m-%d %H:%M:%S.%f') - datetime.strptime(row['cancel_52'], '%Y-%m-%d %H:%M:%S.%f')).total_seconds() for row in result_data]
    threshold = np.percentile(time_diff_values, 90)

    filtered_data = [row for row in result_data if (datetime.strptime(row['cancelack_52'], '%Y-%m-%d %H:%M:%S.%f') - datetime.strptime(row['cancel_52'], '%Y-%m-%d %H:%M:%S.%f')).total_seconds() >= threshold]

    with open(f'/home/elliott/development/files/equity_cancelack_{target}_2%highest_latencies_{date}.csv', 'w') as file:
        fieldnames = ['tag11', 'cancel_52', 'cancelack_52', 'tag41', 'tag100', 'counts']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(filtered_data)

def main():
    parser = argparse.ArgumentParser(description='Run queries and analysis for equity latencies.')
    parser.add_argument('date', help='Date for the queries (YYYY-MM-DD format)')
    parser.add_argument('target', help='Target for the queries')
    parser.add_argument('sender', help='Sender for the queries')

    args = parser.parse_args()

    date = args.date
    target = args.target
    sender = args.sender

    hash_table = run_queries(date, target, sender)
    run_analysis(date, target, hash_table)

if __name__ == '__main__':
    main()



usage: ipykernel_launcher.py [-h] date target sender
ipykernel_launcher.py: error: the following arguments are required: date, target, sender


SystemExit: 2

In [9]:
run_analysis('2023-03-06', 'INCAPNS')

In [8]:
import psycopg2

def run_queries(date, target, sender):    
    conn = psycopg2.connect(
        host="10.7.8.59",
        database="fixtransactions",
        user="scalp",
        password="QAtr@de442",
        port="5433"
    )

    query1 = f"""
    CREATE TEMP TABLE temp_table1 AS
    SELECT tag11, tag100, tag41, tag49 
    FROM fixmsg 
    WHERE tag52 >= TIMESTAMP '{date}'
    AND tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
    AND tag56 = '{target}'
    and tag49 = '{sender}'
    AND (tag35 = 'AB' or tag35 = 'D');
    """

    query2 = f"""
    CREATE TEMP TABLE temp_table2 AS
    SELECT a.tag11, a.tag52 AS cancel_52, b.tag52 AS cancelack_52, b.tag41, a.tag100
    FROM fixmsg a
    JOIN fixmsg b ON a.tag11 = b.tag41
    WHERE a.tag52 >= TIMESTAMP '{date}'
    AND a.tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
    AND b.tag52 >= TIMESTAMP '{date}'
    AND b.tag52 < TIMESTAMP '{date}' + INTERVAL '1 DAY'
    and a.tag49 = '{sender}'
    and b.tag56 = '{sender}'
    and b.tag49 = '{target}'
    AND b.tag35 = '8'
    AND b.tag39 = '4'
    AND (a.tag35 = 'AB' or a.tag35 = 'D')
    AND a.tag56 = '{target}';
    """

    with conn.cursor() as cursor:
        cursor.execute(query1)
        cursor.execute(query2)

        cursor.execute("SELECT * FROM temp_table2")
        query_result = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]

    conn.close()

    hash_table = {}
    for row in query_result:
        row_dict = {column_names[i]: row[i] for i in range(len(column_names))}
        hash_table[row_dict["tag11"]] = row_dict

    return hash_table


In [9]:
run_queries('2023-03-20','INCAPNS','RBTECH3')

{'370040202979420': {'tag11': '370040202979420',
  'cancel_52': datetime.datetime(2023, 3, 20, 13, 30, 56, 470000),
  'cancelack_52': datetime.datetime(2023, 3, 20, 13, 30, 57, 27000),
  'tag41': '370040202979420',
  'tag100': 'PCXOS'},
 '370040202979421': {'tag11': '370040202979421',
  'cancel_52': datetime.datetime(2023, 3, 20, 13, 30, 56, 563000),
  'cancelack_52': datetime.datetime(2023, 3, 20, 13, 30, 57, 118000),
  'tag41': '370040202979421',
  'tag100': 'PCXOS'},
 '370040202979422': {'tag11': '370040202979422',
  'cancel_52': datetime.datetime(2023, 3, 20, 13, 30, 56, 767000),
  'cancelack_52': datetime.datetime(2023, 3, 20, 13, 30, 57, 72000),
  'tag41': '370040202979422',
  'tag100': 'ISEO'},
 '370040202979423': {'tag11': '370040202979423',
  'cancel_52': datetime.datetime(2023, 3, 20, 13, 30, 56, 890000),
  'cancelack_52': datetime.datetime(2023, 3, 20, 13, 30, 57, 415000),
  'tag41': '370040202979423',
  'tag100': 'PCXOS'},
 '370040202979424': {'tag11': '370040202979424',
  